In [1]:
from pyspark.sql.functions import countDistinct
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import *
from pyspark.sql import SQLContext
import json

In [2]:
def read_data(data_source):
    #read the re-partitioned file 
    read_data = spark.read.parquet("gs://ds-url-catag/plenty_stickers_data/"+data_source+"/repartioned_data/*/*/*/*.parquet")
    #dropduplicates
    read_data = read_data.dropDuplicates()
    return read_data

def extract_columns(dataframe,data_source):
    #function to split a single column to multiple columns
    def extract_json(line, schema_names):
        result = [x for i,x in enumerate(line) if i!=4]
        properties = json.loads(line[4])
        flattened = []     
        for key in schema_names:
            if key in properties:
                if type(properties[key]) == unicode :
                    flattened.append(properties[key].encode('utf-8'))
                else:
                    flattened.append(str(properties[key]))         
            else:
                flattened.append("NULL")
        result = list(result) + flattened
        return result
    json_schema_send = spark.read.json(dataframe.rdd.map(lambda row: row.properties)).schema
    #creating the new columns for the file 
    old_column_names_send = dataframe.drop('properties').columns
    new_items_send = ['user_id_n' if x == 'user_id' else 'app_version_n' if  x == 'app_version' else 'city_n' if  x == 'city' else 'connection_n' if  x == 'connection' else 'country_n' if  x == 'country' else 'device_brand_n' if  x == 'device_brand' else 'device_model_n' if  x == 'device_model' else 'has_nfc_n' if  x == 'has_nfc' else 'height_resolution_n' if  x == 'height_resolution' else 'new_user_n' if  x == 'new_user' else 'new_user_n_1' if  x == 'new_User' else 'os_version_n' if  x == 'os_version' else 'width_resolution_n' if  x == 'width_resolution' else x for x in json_schema_send.names]
    fin_col_names_send = old_column_names_send + new_items_send
    df = dataframe.rdd.map(lambda x: extract_json(x, json_schema_send.names)).toDF(fin_col_names_send)
    df = df.withColumn('City',when(df.City == 'NULL' ,df.city_n).otherwise(df.City))
    df = df.drop('city_n')
    df = df.dropDuplicates()
    df = df.repartition(200)
    print "writing to files"
    #write to new files after processing for BBM-Sticker-Send
    if data_source == 'event=BBM-STICKER-DOWNLOAD':
        df.select('id',
         'visit_id',
         'time',
         'City',
         'sticker_pack_id',
         'user_id_n').dropDuplicates().write.mode('overwrite').parquet("gs://ds-url-catag/plenty_stickers_data/"+data_source+"/aggregate_data/")
    else:
        df.select('id',
         'visit_id',
         'time',
         'City',
         'sticker_id',
         'sticker_pack_id',
         'user_id_n').dropDuplicates().write.mode('overwrite').parquet("gs://ds-url-catag/plenty_stickers_data/"+data_source+"/aggregate_data/")
    return "data written"

In [6]:
if __name__ == "__main__":
    Sticker_datasources = ['event=BBM-STICKER-DOWNLOAD']
    #values for data sources
    #'event=BBM-STICKER-RECEIVED','event=BBM-STICKER_SPONSORED_LINK-CLICK','event=BBM-STICKER-CLICK','event=BBM-STICKER-DOWNLOAD','event=BBM-STICKER-SEND'
    for ds in Sticker_datasources:
        dataframe  = read_data(ds)
        extract_columns(dataframe,ds)